# Module 1. Check Inference Results & Local Mode Deployment
---

## Overview

본 핸즈온은 AWS AIML Blog의 내용을 기반으로 MNIST 예제 대신 좀 더 실용적인 한국어 자연어 처리 예시를 다루며, 총 3종류(Sentiment Classification, KorSTS, KoBART)의 자연어 처리 모델을 SageMaker 다중 컨테이너 엔드포인트(Multi-container endpoint)로 배포하는 법을 익혀 봅니다.

이미 SageMaker 기본 개념(로컬 모드, 호스팅 엔드포인트)과 자연어 처리 & Huggingface을 다뤄 보신 분들은 이 섹션을 건너 뛰고 다음 노트북으로 진행하셔도 됩니다.

### References
- AWS AIML Blog: https://aws.amazon.com/ko/blogs/machine-learning/deploy-multiple-serving-containers-on-a-single-instance-using-amazon-sagemaker-multi-container-endpoints/
- Developer Guide: https://docs.aws.amazon.com/sagemaker/latest/dg/multi-container-endpoints.html

In [ ]:
!pip install -qU sagemaker botocore boto3 awscli
!pip install --ignore-installed PyYAML
!pip install transformers==4.12.5

In [ ]:
import json
import os
import sys
import torch
import boto3
import sagemaker
import datetime
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from src.utils import print_outputs, prepare_model_artifact, NLPPredictor 

role = get_execution_role()
boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session()
sm_client = boto_session.client("sagemaker")
sm_runtime = boto_session.client("sagemaker-runtime")

<br>

## 1. Check Inference Results & Debugging
---

로컬 엔드포인트나 호스팅 엔드포인트 배포 전, 로컬 환경 상에서 직접 추론을 수행하여 결과를 확인합니다. 참고로, SageMaker에서 TensorFlow를 제외한 머신 러닝 프레임워크 추론 컨테이너는 아래의 인터페이스를 사용합니다.

#### Option 1.
- `model_fn(model_dir)`: 네트워크 아키텍처를 정의하고 S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `input_fn(request_body, content_type)`: 입력 데이터를 전처리합니다. (예: request_body로 전송된 bytearray 배열을 PIL.Image로 변환 수 cropping, resizing, normalization등의 전처리 수행). content_type은 입력 데이터 종류에 따라 다양하게 처리 가능합니다. (예: application/x-npy, application/json, application/csv 등)
- `predict_fn(input_object, model)`: input_fn을 통해 들어온 데이터에 대해 추론을 수행합니다.
- `output_fn(prediction, accept_type)`: predict_fn에서 받은 추론 결과를 추가 변환을 거쳐 프론트 엔드로 전송합니다.

#### Option 2.
- `model_fn(model_dir)`: 네트워크 아키텍처를 정의하고 S3의 model_dir에 저장된 모델 아티팩트를 로드합니다.
- `transform_fn(model, request_body, content_type, accept_type)`: input_fn(), predict_fn(), output_fn()을 transform_fn()으로 통합할 수 있습니다.

모델, 배포에 초점을 맞추기 위해 Huggingface에 등록된 `KoELECTRA-Small-v3` 모델을 기반으로 네이버 영화 리뷰 데이터셋과 KorSTS (Korean Semantic Textual Similarity) 데이터셋으로 파인 튜닝하였습니다. 파인 튜닝은 온프레미스나 Huggingface on SageMaker로 쉽게 수행 가능합니다. 

- KoELECTRA: https://github.com/monologg/KoELECTRA
- Huggingface on Amazon SageMaker: https://huggingface.co/docs/sagemaker/main


### Model A: Sentiment Classification

네이버 영화 리뷰 데이터의 긍정/부정 판별 예시입니다. 
- Naver sentiment movie corpus: https://github.com/e9t/nsmc
- 예시
    - '이 영화는 최고의 영화입니다' => {"predicted_label": "Pos", "score": 0.96}
    - '최악이에요. 배우의 연기력도 좋지 않고 내용도 너무 허접합니다' => {"predicted_label": "Neg", "score": 0.99}

In [ ]:
!pygmentize src/inference_nsmc.py

In [ ]:
from src.inference_nsmc import model_fn, input_fn, predict_fn, output_fn
modelA_path = 'model-nsmc'

with open('samples/nsmc.txt', mode='rb') as file:
    modelA_input_data = file.read()

modelA = model_fn(modelA_path)
transformed_inputs = input_fn(modelA_input_data)
predicted_classes_jsonlines = predict_fn(transformed_inputs, modelA)
modelA_outputs = output_fn(predicted_classes_jsonlines)
print(modelA_outputs[0])    

### Model B: Semantic Textual Similarity (STS)

두 문장간의 유사도를 정량화하는 예시입니다.
- KorNLI and KorSTS: https://github.com/kakaobrain/KorNLUDatasets
- 예시
    - ['맛있는 라면을 먹고 싶어요', '후루룩 쩝쩝 후루룩 쩝쩝 맛좋은 라면'] => {"score": 4.78}
    - ['뽀로로는 내친구', '머신러닝은 러닝머신이 아닙니다.'] => {"score": 0.23}

In [ ]:
!pygmentize src/inference_korsts.py

In [ ]:
from src.inference_korsts import model_fn, input_fn, predict_fn, output_fn
modelB_path = 'model-korsts'

with open('samples/korsts.txt', mode='rb') as file:
    modelB_input_data = file.read()    
    
modelB = model_fn(modelB_path)
transformed_inputs = input_fn(modelB_input_data)
predicted_classes_jsonlines = predict_fn(transformed_inputs, modelB)
modelB_outputs = output_fn(predicted_classes_jsonlines)
print(modelB_outputs[0])

### Model C: KoBART (Korean Bidirectional and Auto-Regressive Transformers)

문서 내용(예: 뉴스 기사)을 요약하는 예시입니다.

- KoBART: https://github.com/SKT-AI/KoBART
- KoBART Summarization: https://github.com/seujung/KoBART-summarization

In [ ]:
!pygmentize src/inference_kobart.py

S3로 모델 아티팩트를 복사하는 대신 Huggingface에 등록된 모델을 그대로 사용합니다. model.pth는 0바이트의 빈 파일이며, 추론을 수행하기 위한 소스 코드들만 아카이빙됩니다.

In [ ]:
from src.inference_kobart import model_fn, transform_fn
modelC_path = 'model-kobart'
f = open(f"{modelC_path}/model.pth", 'w')
f.close()

with open('samples/kobart.txt', mode='rb') as file:
    modelC_input_data = file.read()

modelC = model_fn('./')
outputs = transform_fn(modelC, modelC_input_data)

with open('samples/kobart.txt', mode='rb') as file:
    modelC_input_data = file.read()

결괏값들을 확인했다면 로컬 모드로 빠르게 배포하여 테스트하는 것을 권장드립니다. 단, SageMaker Studio는 로컬 모드를 지원하지 않기 때문에 아래 섹션은 SageMaker에서 실행해 주세요.

<br>

## 2. (SageMaker Only) Local Mode Deployment for Model A
---

### Deploy Model A

In [ ]:
modelA_artifact_name = 'modelA.tar.gz'
prepare_model_artifact(modelA_path, model_artifact_name=modelA_artifact_name)
local_model_path = f'file://{os.getcwd()}/{modelA_artifact_name}'

model = PyTorchModel(
    model_data=local_model_path,
    role=role,
    entry_point='inference_nsmc.py', 
    source_dir='src',
    framework_version='1.7.1',
    py_version='py3',
    predictor_cls=NLPPredictor,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local'
)

### Invoke using SageMaker Python SDK
SageMaker SDK `predict()` 메서드로 간단하게 추론을 실행할 수 있습니다. 

In [ ]:
inputs = [{"text": ["이 영화는 최고의 영화입니다"]}, 
          {"text": ["최악이에요. 배우의 연기력도 좋지 않고 내용도 너무 허접합니다"]}]

predicted_classes = predictor.predict(inputs)

In [ ]:
for c in predicted_classes:
    print(c)

### Invoke using Boto3 API
이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리 SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다. 

In [ ]:
local_sm_runtime = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = model.endpoint_name

response = local_sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/jsonlines',
    Accept='application/jsonlines',
    Body=modelA_input_data
    )
outputs = response['Body'].read().decode()               

In [ ]:
print_outputs(outputs) 

### Local Mode Endpoint Clean-up
엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다.

로컬 엔드포인트는 도커 컨테이너이기 때문에 `docker rm $(docker ps -a -q)` 으로도 간단히 삭제할 수 있습니다.

In [ ]:
predictor.delete_endpoint()

<br>

## 3. (SageMaker Only) Local Mode Deployment for Model B
---

### Deploy Model B

In [ ]:
modelB_artifact_name = 'modelB.tar.gz'
prepare_model_artifact(modelB_path, model_artifact_name=modelB_artifact_name)
local_model_path = f'file://{os.getcwd()}/{modelB_artifact_name}'

model = PyTorchModel(
    model_data=local_model_path,
    role=role,
    entry_point='inference_korsts.py', 
    source_dir='src',
    framework_version='1.7.1',
    py_version='py3',
    predictor_cls=NLPPredictor,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local'
)

### Invoke using SageMaker Python SDK
SageMaker SDK `predict()` 메서드로 간단하게 추론을 실행할 수 있습니다. 

In [ ]:
inputs = [{"text": ["맛있는 라면을 먹고 싶어요", "후루룩 쩝쩝 후루룩 쩝쩝 맛좋은 라면"]}, 
          {"text": ["뽀로로는 내친구", "머신러닝은 러닝머신이 아닙니다."]}]

predicted_classes = predictor.predict(inputs)

In [ ]:
for c in predicted_classes:
    print(c)

### Invoke using Boto3 API
이번에는 boto3의 `invoke_endpoint()` 메서드로 추론을 수행해 보겠습니다.
Boto3는 서비스 레벨의 low-level SDK로, ML 실험에 초점을 맞춰 일부 기능들이 추상화된 high-level SDK인 SageMaker SDK와 달리 SageMaker API를 완벽하게 제어할 수 있습으며, 프로덕션 및 자동화 작업에 적합합니다. 

In [ ]:
local_sm_runtime = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = model.endpoint_name

response = local_sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/jsonlines',
    Accept='application/jsonlines',
    Body=modelB_input_data
    )
outputs = response['Body'].read().decode()               

In [ ]:
print_outputs(outputs)

### Local Mode Endpoint Clean-up
엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다.

로컬 엔드포인트는 도커 컨테이너이기 때문에 `docker rm $(docker ps -a -q)` 으로도 간단히 삭제할 수 있습니다.

In [ ]:
predictor.delete_endpoint()

<br>

## 4. (SageMaker Only) Local Mode Deployment for Model C
---

### Deploy Model C

In [ ]:
modelC_artifact_name = 'modelC.tar.gz'
prepare_model_artifact(modelC_path, model_artifact_name=modelC_artifact_name)
local_model_path = f'file://{os.getcwd()}/{modelC_artifact_name}'

model = PyTorchModel(
    model_data=local_model_path,
    role=role,
    entry_point='inference_kobart.py', 
    source_dir='src',
    framework_version='1.7.1',
    py_version='py3',
    predictor_cls=NLPPredictor,
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type='local'
)

In [ ]:
import time
time.sleep(3)

### Invoke using Boto3 API
**[주의]** BART 모델은 Auto-Regressive 모델로 내부적으로 연산을 많이 수행하여 기본 인스턴스(예: `ml.t2.medium`)를 사용하는 경우, 시간이 상대적으로 오래 소요됩니다.

In [ ]:
local_sm_runtime = sagemaker.local.LocalSagemakerRuntimeClient()
endpoint_name = model.endpoint_name

response = local_sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='application/jsonlines',
    Accept='application/jsonlines',
    Body=modelC_input_data
    )
outputs = response['Body'].read().decode()             

In [ ]:
print_outputs(outputs)

<br>

## Local Mode Endpoint Clean up
---

엔드포인트를 계속 사용하지 않는다면, 엔드포인트를 삭제해야 합니다. SageMaker SDK에서는 `delete_endpoint()` 메소드로 간단히 삭제할 수 있습니다.
참고로, 노트북 인스턴스에서 추론 컨테이너를 배포했기 때문에 엔드포인트를 띄워 놓아도 별도로 추가 요금이 과금되지는 않습니다.

로컬 엔드포인트는 도커 컨테이너이기 때문에 `docker rm $(docker ps -a -q)` 으로도 간단히 삭제할 수 있습니다.

In [ ]:
predictor.delete_endpoint()